In [1]:
import tensorflow as tf
print(tf.__version__)

C:\Users\MSALMANN\AppData\Roaming\Python\Python38\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


2.8.0


In [2]:
# More importing common libraries

from tensorflow.keras.layers import Input, Dense, LeakyReLU, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD, Adam

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys, os

In [3]:
# Load the MNIST Data

mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Scale the inputs in range of (-1, +1) for better training
x_train, x_test = x_train / 255.0 * 2 - 1, x_test / 255.0 * 2 - 1

In [4]:
print ("x_train.shape:" ,x_train.shape)
print ("x_test.shape:" ,x_test.shape)

x_train.shape: (60000, 28, 28)
x_test.shape: (10000, 28, 28)


In [5]:
# Flatten the data
N, H, W = x_train.shape
D = H * W
x_train = x_train.reshape(-1, D)
x_test = x_test.reshape(-1, D)

In [6]:
print ("x_train.shape:" ,x_train.shape)
print ("x_test.shape:" ,x_test.shape)

x_train.shape: (60000, 784)
x_test.shape: (10000, 784)


In [7]:
# Dimensionality of the latent space
latent_dim = 100

In [8]:
# Defining the generator model
def build_generator(latent_dim):
    i = Input(shape=(latent_dim,))
    x = Dense(256, activation=LeakyReLU(alpha=0.2))(i)
    x = BatchNormalization(momentum=0.7)(x)
    x = Dense(512, activation=LeakyReLU(alpha=0.2))(x)
    x = BatchNormalization(momentum=0.7)(x)
    x = Dense(1024, activation=LeakyReLU(alpha=0.2))(x)
    x = BatchNormalization(momentum=0.7)(x)
    x = Dense(D, activation='tanh')(x)

    model = Model(i, x)
    return model

In [9]:
# Defining the discriminator model
def build_discriminator(img_size):
    i = Input(shape=(img_size,))
    x = Dense(512, activation=LeakyReLU(alpha=0.2))(i)
    x = Dense(256, activation=LeakyReLU(alpha=0.2))(x)
    x = Dense(1, activation='sigmoid')(x)


    model = Model(i, x)
    return model

In [10]:
# Compile both models in preparation for training

# Build and compile the discriminator
discriminator = build_discriminator(D)
discriminator.compile ( loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5), metrics=['accuracy'])

# Build and compile the combined model
generator = build_generator(latent_dim)

In [11]:
generator.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 100)]             0         
                                                                 
 dense_3 (Dense)             (None, 256)               25856     
                                                                 
 batch_normalization (BatchN  (None, 256)              1024      
 ormalization)                                                   
                                                                 
 dense_4 (Dense)             (None, 512)               131584    
                                                                 
 batch_normalization_1 (Batc  (None, 512)              2048      
 hNormalization)                                                 
                                                                 
 dense_5 (Dense)             (None, 1024)              5253

In [12]:
discriminator.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 784)]             0         
                                                                 
 dense (Dense)               (None, 512)               401920    
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dense_2 (Dense)             (None, 1)                 257       
                                                                 
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 0
_________________________________________________________________


In [13]:
# Create an input to represent noise sample from latent space
z = Input(shape=(latent_dim,))

z.shape

TensorShape([None, 100])

In [14]:
# Pass noise through generator to get an image
img = generator(z)

img

<KerasTensor: shape=(None, 784) dtype=float32 (created by layer 'model_1')>

In [15]:
# Make sure only the generator is trained
discriminator.trainable = False

In [16]:
# The true output is fake, but we label them real!
# Passing the output of Generator to the Discriminator

fake_pred = discriminator(img)

In [17]:
# Create the combined model object
combined_model_gen = Model(z, fake_pred)

In [18]:
# Compile the combined model
combined_model_gen.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5))

In [19]:
# Config

batch_size = 32
epochs = 30000
sample_period = 200 # every `sample_period` steps generate and save some data"

In [20]:
# Create batch labels to use when calling train_on_batch
ones = np.ones(batch_size)
zeros = np.zeros(batch_size)

# Store the losses
d_losses = []
g_losses = []

# Create a folder to store generated images
if not os.path.exists('gan_images'):
    os.makedirs('gan_images')

In [21]:
# A function to generate a grid of random samples from the generator and save them to a file

def sample_images(epoch):
    rows, cols = 5, 5
    noise = np.random.randn(rows * cols, latent_dim)
    imgs = generator.predict(noise)

    # Rescale images 0 - 1
    imgs = 0.5 * imgs + 0.5

    fig, axs = plt.subplots(rows, cols)
    idx = 0
    for i in range(rows):
        for j in range(cols):
            axs[i,j].imshow(imgs[idx].reshape(H, W), cmap='gray')
            axs[i,j].axis('off')
            idx += 1
    fig.savefig("gan_images/%d.png" % epoch)
    plt.close()

In [22]:
# Main training loop
for epoch in range(epochs):
  ###########################
  ### Train discriminator ###
  ###########################
  
  # Select a random batch of images
    idx = np.random.randint(0, x_train.shape[0], batch_size)
    real_imgs = x_train[idx]

    # Generate fake images
    noise = np.random.randn(batch_size, latent_dim)
    fake_imgs = generator.predict(noise)

    # Train the discriminator
    # both loss and accuracy are returned
    d_loss_real, d_acc_real = discriminator.train_on_batch(real_imgs, ones)
    d_loss_fake, d_acc_fake = discriminator.train_on_batch(fake_imgs, zeros)
    d_loss = 0.5 * (d_loss_real + d_loss_fake)
    d_acc  = 0.5 * (d_acc_real + d_acc_fake)


    #######################
    ### Train generator ###
    #######################

    noise = np.random.randn(batch_size, latent_dim)
    g_loss = combined_model_gen.train_on_batch(noise, ones)

    # do it again!
    noise = np.random.randn(batch_size, latent_dim)
    g_loss = combined_model_gen.train_on_batch(noise, ones)

    # Save the losses
    d_losses.append(d_loss)
    g_losses.append(g_loss)

    if epoch % 100 == 0:
        print(f"epoch: {epoch+1}/{epochs}, d_loss: {d_loss:.2f}, d_acc: {d_acc:.2f}, g_loss: {g_loss:.2f}")

    if epoch % sample_period == 0:
        sample_images(epoch)

epoch: 1/30000, d_loss: 0.81, d_acc: 0.27, g_loss: 0.47
epoch: 101/30000, d_loss: 0.03, d_acc: 1.00, g_loss: 3.74
epoch: 201/30000, d_loss: 1.10, d_acc: 0.28, g_loss: 0.50
epoch: 301/30000, d_loss: 0.69, d_acc: 0.45, g_loss: 0.63
epoch: 401/30000, d_loss: 0.68, d_acc: 0.47, g_loss: 0.62
epoch: 501/30000, d_loss: 0.68, d_acc: 0.50, g_loss: 0.66
epoch: 601/30000, d_loss: 0.69, d_acc: 0.45, g_loss: 0.67
epoch: 701/30000, d_loss: 0.66, d_acc: 0.48, g_loss: 0.68
epoch: 801/30000, d_loss: 0.68, d_acc: 0.45, g_loss: 0.69
epoch: 901/30000, d_loss: 0.67, d_acc: 0.59, g_loss: 0.71
epoch: 1001/30000, d_loss: 0.70, d_acc: 0.47, g_loss: 0.71
epoch: 1101/30000, d_loss: 0.69, d_acc: 0.50, g_loss: 0.73
epoch: 1201/30000, d_loss: 0.70, d_acc: 0.42, g_loss: 0.77
epoch: 1301/30000, d_loss: 0.66, d_acc: 0.56, g_loss: 0.80
epoch: 1401/30000, d_loss: 0.68, d_acc: 0.53, g_loss: 0.76
epoch: 1501/30000, d_loss: 0.59, d_acc: 0.75, g_loss: 0.81
epoch: 1601/30000, d_loss: 0.66, d_acc: 0.56, g_loss: 0.77
epoch: 17

In [23]:
# data = generator.predict(noise[1].reshape(1, noise[0].shape[0])).reshape(28,28)

In [24]:
# data = generator.predict(np.random.rand(1,100)).reshape(28,28)
# plt.imshow(data, interpolation='nearest')
# plt.show()

In [26]:
# generator.save('generator_model.h5')